In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
!cp -r /content/drive/MyDrive/v8_format/* .

In [ ]:
# !rsync -a /content/drive/MyDrive/v8_format .

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=data.yaml epochs=50 imgsz=640 batch=8 project=/content/drive/MyDrive/ball_output name=Ball_Detection

In [ ]:
# !yolo task=detect mode=train model=yolov8s.pt data=data.yaml epochs=50 imgsz=640 batch=8 project=/content/drive/MyDrive/ball_output name=Ball_Detection_1 optimizer='AdamW' close_mosaic=10 cos_lr=True degrees: 0.2 shear: 0.2 perspective: 0.01 mixup: 0.3 warmup_epochs=1.0

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/ball_output/Ball_Detection/weights/best.pt conf=0.55 source=/content/drive/MyDrive/v8_format/images/val/002af65597d93000.jpg save=true

In [ ]:
!pip install supervision

In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv
import numpy as np

# --- Configuration ---
# Note: Make sure these paths are correct for your Colab environment.
input_video_path = "/content/drive/MyDrive/29_sept_IITD/foot_ball.mp4"
output_video_path = "/content/drive/MyDrive/29_sept_IITD/detection_output.mp4"
model_path = "/content/drive/MyDrive/ball_output/Ball_Detection/weights/best.pt"

# --- Video Capture ---
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print(f"Error: Could not open video file at {input_video_path}")
else:
    # --- Get Video Properties ---
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # --- Video Writer ---
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # --- Model and Annotators ---
    model = YOLO(model_path)

    # Create a BoxAnnotator for drawing boxes ONLY
    box_annotator = sv.BoxAnnotator(
        thickness=2
    )

    # Create a LabelAnnotator for drawing text labels ONLY
    label_annotator = sv.LabelAnnotator(
        text_thickness=2,
        text_scale=1
    )

    print("🚀 Starting video processing...")

    # --- Main Loop ---
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        result = model(frame, agnostic_nms=True)[0]
        detections = sv.Detections.from_ultralytics(result)

        # CORRECTED LINE: Access class_id and confidence directly from detections
        labels = [
            f"{model.model.names[class_id]} {confidence:0.2f}"
            for class_id, confidence
            in zip(detections.class_id, detections.confidence)
        ]

        # --- Two-step annotation process ---
        # 1. Annotate the frame with bounding boxes
        frame = box_annotator.annotate(
            scene=frame,
            detections=detections
        )

        # 2. Annotate the frame with labels on top of the boxes
        frame = label_annotator.annotate(
            scene=frame,
            detections=detections,
            labels=labels
        )

        out.write(frame)

    print(f"✅ Processing complete. Video saved to {output_video_path}")

    # --- Cleanup ---
    cap.release()
    out.release()
    cv2.destroyAllWindows()